In [1]:
# the imports in this cell are required when running from Cloud (Colab/Kaggle)
# before running on cloud you nee to upload the .py files 
# from 'Notebooks/utils' directory
from applyML_util import train_regression, eval_regression, showEvalutationGraph_regression
from featureSelection_util import (pearson_correlation_fs, 
                                   seleckKBest_fs, selectSequential_fs)

# New Section

**Random Forest Documentation link:** https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

**LGBMC Documentation link:** https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html

**SK-Lego Documentation link:** https://scikit-lego.netlify.app/meta.html#Zero-Inflated-Regressor

In [ ]:
!pip install sklego

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestRegressor
from sklego.meta import ZeroInflatedRegressor #!pip install sklego


In [4]:
# global random seed
RAND_SEED = 42

ziReg = ZeroInflatedRegressor(classifier=LGBMClassifier(random_state=RAND_SEED), regressor=RandomForestRegressor(random_state=RAND_SEED, n_jobs=-1))

# initial model with only random seed and not any hyper-parametes
initial_model = ziReg

# hyper-parameters
n_estimators = [x*5 for x in range(15, 41)] 
max_features = ['sqrt']
learning_rate = [0.01,0.1]

param_grid = {'classifier__learning_rate' : learning_rate,
              'classifier__n_estimators': n_estimators,
              'regressor__max_features': max_features,
              'regressor__n_estimators': n_estimators
}

# # variables needed for showEvalGraph_regression() function
# MODEL_CLASS = ziReg
# x_axis_param_name = 'regressor__n_estimators'
# x_axis_vals = n_estimators

## 1. Experimentation on the Weather Daily dataset

In [5]:
# Load the train dataset
weather_daily_train_df = pd.read_csv('https://raw.githubusercontent.com/ferdouszislam/Weather-WaterLevel-Prediction-ML/main/Datasets/brri-datasets/final-dataset/train/brri-weather_train_regression.csv')

# Load the test set
weather_daily_test_df = pd.read_csv('https://raw.githubusercontent.com/ferdouszislam/Weather-WaterLevel-Prediction-ML/main/Datasets/brri-datasets/final-dataset/test/brri-weather_test_regression.csv')

In [ ]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

Selected hyperparameters: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 185, 'regressor__max_features': 'sqrt', 'regressor__n_estimators': 155}
Train set performance: r2-score=0.2313, mae=5.5913, rmse=13.0461


In [ ]:
# # r2-scores graph on the train set

# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [ ]:
# test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')

Test set performance: r2-score=0.1471, mae=5.983, rmse=15.9686


### 1.1 Apply Pearson Feature Selection to Daily Weather Dataset

In [ ]:
# select features from the train dataset
weather_daily_fs1_train_df, cols_to_drop = pearson_correlation_fs(weather_daily_train_df, 'Rainfall (mm)')

# keep only selected features on the test dataset
weather_daily_fs1_test_df = weather_daily_test_df.drop(columns=cols_to_drop)

dropping Sunshine (hour/day) from (Cloudy (hour/day), Sunshine (hour/day))


In [ ]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_fs1_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

Selected hyperparameters: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 200, 'regressor__max_features': 'sqrt', 'regressor__n_estimators': 200}
Train set performance: r2-score=0.2152, mae=5.5988, rmse=13.1739


In [ ]:
# # r2-scores graph on the train set

# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_fs1_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [ ]:
# test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_fs1_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')

Test set performance: r2-score=0.1511, mae=5.8962, rmse=15.9308


### 1.2 Apply SelectKBest Feature Selection to Daily Weather Dataset

In [ ]:
# select features from the train dataset
weather_daily_fs2_train_df, cols_to_drop = seleckKBest_fs(weather_daily_train_df, 'Rainfall (mm)', is_regression=True)

print('features dropped:', cols_to_drop)

# keep only selected features on the test dataset
weather_daily_fs2_test_df = weather_daily_test_df.drop(columns=cols_to_drop)

features dropped: ['Max Temp. (degree Celcius)', 'Solar Radiation (cal/cm^2/day)']


In [ ]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_fs2_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

Selected hyperparameters: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 200, 'regressor__max_features': 'sqrt', 'regressor__n_estimators': 175}
Train set performance: r2-score=0.2287, mae=5.5097, rmse=13.0941


In [ ]:
# # r2-scores graph on the train set

# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_fs2_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [ ]:
# test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_fs2_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')

### 1.3 Apply SelectSequential Feature Selection to Daily Weather Dataset

In [ ]:
# select features from the train dataset
weather_daily_fs3_train_df, cols_to_drop = selectSequential_fs(weather_daily_train_df, 'Rainfall (mm)', is_regression=True)

print('features dropped:', cols_to_drop)

# keep only selected features on the test dataset
weather_daily_fs3_test_df = weather_daily_test_df.drop(columns=cols_to_drop)

In [ ]:
# train model
model, selected_hyperparams, train_r2, train_mae, train_rmse = train_regression(initial_model, param_grid, weather_daily_fs3_train_df, cls='Rainfall (mm)')
print(f'Selected hyperparameters: {selected_hyperparams}')
# performance on the train set
print(f'Train set performance: r2-score={train_r2}, mae={train_mae}, rmse={train_rmse}')

In [ ]:
# # r2-scores graph on the train set

# # hyper-parameters selected by GridSearchCV
# selected_model_params = selected_hyperparams
# #selected_model_params['random_state'] = RAND_SEED

# showEvalutationGraph_regression(MODEL_CLASS, weather_daily_fs3_train_df, cls='Rainfall (mm)', 
#                                 x_axis_param_name=x_axis_param_name, x_axis_param_vals=x_axis_vals, 
#                                 selected_model_params=selected_model_params)

In [ ]:
# # test model
test_r2, test_mae, test_rmse = eval_regression(model, weather_daily_fs3_test_df, cls='Rainfall (mm)')
# performance on the test set
print(f'Test set performance: r2-score={test_r2}, mae={test_mae}, rmse={test_rmse}')